<a href="https://colab.research.google.com/github/JackEdwardAuty/Markdown-Notes-Processing-and-Analysis/blob/main/Markdown_Processing_Python_Personal_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Markdown Processing Personal Project
## Introduction
Over the past few years I have been using the popular Markdown editor, Obsidian, to keep track of medically useful information about myself. This has been useful to check for example whether I have taken my medication today or when I last ate.

The usefulness of the Obsidian Vault~

capitalise on the skills learnt through this Just-IT Data Skills Bootcamp in order to consolidate the source dataset, extracting three key tables (medication, diet, sleep quality) of consolidated information for further analysis.

- Extract
- Transform
- Load
- Verify (Integrity + Validity)
- can analyse how long from serving did I start eating, how long to finish (any relationship between longer between serving and starting, and length taken)


### Extract
There are over 1500 markdown files in my `'mobile-notes-dataset'` from which to extract data, My process and approach has evolved since the beginning, and thus there are different templates used and information to extract between different time periods. This means that not only will I have to implement a specific data extraction script for each area data is tracked for, but that those scripts must be able to handle the different forms used between templates without breaking or giving faulty data.

### Transform & Load
Once the data has been succesfully extracted from the 1500~ markdown files into just three csv files, `medication.csv`, `wake_time.csv` and `dietary.csv`, these tables can then be imported into either Excel or PowerBI for transform the extracted data into a usable form.
Some of the transformations that I will need to do include:
- filter/delete rows with blank/null values for essential fields
- handle non-standard inputs for `time` field and `dosage` field in medication table
  - `time` field contains multiple csv delimmited times
    - duplicate row for each time listed
    - ensure duplicated rows have correct dosages, sometimes 1/2 or 3/2 initial
  - `dosage` field contains units, non-standard measurements such as `1 tablet`
    - handle dosage multiplication such that only numerical value multiplied
    - handle units, either pull into column if can consolidate or find other solution
- continously verify data validity and integrity against expectations, until all non-standard values are handled
- consider filling in missing meal times using images
  - consider how to handle meals with only times vs ones with all ingredients

At this stage it will become clear what other transformations need to be carried out before Loading the data into PowerBI or Tableau for visualisation purposes.


## Loading (then extracting) the Data from the markdown `mobile-notes-dataset`
The first step in my project is to load the markdown files from their non-standard folder structure, handling duplicates and empty templates, ensuring all files in the dataset are utilised and imported for use in further programmatic handling.
### File Handling Approach


In [ ]:
import os
import re
import pandas as pd

# Dataset Directory
mobileNotesDirectory = 'path/to/mobile-notes-dataset' # All of my markdown files in various subdirectories

# List and Filter the markdown files
#either
def list_md_files(folder):
    return [f for f in os.listdir(folder)
            if f.endswith('.md') and not f.startswith('template')]

#or
from pathlib import Path

def list_md_files(folder):
    folder_path = Path(folder)
    return [f for f in folder_path.iterdir()
            if f.is_file() and f.suffix == '.md' and not f.name.startswith('template')]

# Iterate through markdown files for extraction
for file in list_md_files():
    full_path = os.path.join(meds_folder, file)
    # open file, parse contents, extract meds/wake info

# Date Extraction and Folder Context - capture folder name as date context while processing
from pathlib import Path

def extract_date_from_path(file_path):
    path_obj = Path(file_path)
    # assuming folder structure meds_folder/YYYY-MM/filename.md
    folder_date = path_obj.parent.name
    return folder_date

# Reading File Contents
# open files safely with encoding as needed
with open(full_path, 'r', encoding='utf-8') as f:
    contents = f.read()
# Process contents with regex or parsing logic to extract meds/wake/diet times.


NameError: name 'meds_folder' is not defined

In [ ]:
import os
import re
import pandas as pd

# Dataset Directory
mobileNotesDirectory = 'path/to/mobile-notes-dataset' # All of my markdown files in various subdirectories

# List and Filter the markdown files
#either
def list_md_files(folder):
    return [f for f in os.listdir(folder)
            if f.endswith('.md') and not f.startswith('template')]

#or
from pathlib import Path

def list_md_files(folder):
    folder_path = Path(folder)
    return [f for f in folder_path.iterdir()
            if f.is_file() and f.suffix == '.md' and not f.name.startswith('template')]

# Iterate through markdown files for extraction
for file in list_md_files(mobileNotesDirectory):
    full_path = os.path.join(mobileNotesDirectory, file)
    print(full_path)
    # open file, parse contents, extract meds/wake info

# Date Extraction and Folder Context - capture folder name as date context while processing
from pathlib import Path

def extract_date_from_path(file_path):
    path_obj = Path(file_path)
    # assuming folder structure meds_folder/YYYY-MM/filename.md
    folder_date = path_obj.parent.name
    return folder_date

# Reading File Contents
# open files safely with encoding as needed
with open(full_path, 'r', encoding='utf-8') as f:
    contents = f.read()
# Process contents with regex or parsing logic to extract meds/wake/diet times.


FileNotFoundError: [Errno 2] No such file or directory: 'path/to/mobile-notes-dataset'

## Extracting the Wake Times from dataset
extracting the useful information, file by file using regex

In [ ]:
def extract_wake_times_from_folder(folder_path):
    wakes = []
    for filename in os.listdir(folder_path):
        if not filename.endswith('.md'):
            continue
        full_path = os.path.join(folder_path, filename)
        # Extract wake time info similarly with parsing, timestamp normalization etc.
    return pd.DataFrame(wakes)